# DFT: The LDA kernel
## I. Theory

Previously we described the DFT Fock matrix as
$$F^{DFT}_{\mu\nu} = H_{\mu\nu} + 2J[D]_{\mu\nu} - \zeta K[D]_{\mu\nu} + V^{\rm{xc}}_{\mu\nu}$$
upon examination it is revealed that the only quantities that we cannot yet compute is $V^{\rm{xc}}$. 

Here we will explore the local density approximation (LDA) functionals where $V^{\rm{xc}} = f[\rho(\hat{r})]$. For these functionals the only required bit of information is the density at the grid point. As we discussed the grid last chapter we will now focus on how exactly to obtain the density on the grid.

In [1]:
import psi4
import numpy as np

mol = psi4.geometry("""
He
symmetry c1
""")
psi4.set_options({'BASIS':               'CC-PVDZ',
                  'DFT_SPHERICAL_POINTS': 6,
                  'DFT_RADIAL_POINTS':    5})

svwn_w, wfn = psi4.energy("SVWN", return_wfn=True)
Vpot = wfn.V_potential()

## 2. Density on a Grid
The density on the grid can be expressed as
$$\rho^{\alpha}(\hat{r}) = \sum\limits_{\mu\nu} D^\alpha_{\mu\nu}\phi_\mu(\hat{r})\phi_\nu(\hat{r})$$

Where $\hat{r}$ will run over the entire grid. Using this we can build collocation matrices that map between atomic orbital and grid space $$\phi_\mu(\hat{r}) \rightarrow \phi_\mu^p$$
where our $p$ index will be the index of individual grid points. Therefore our full expression is:

$$\rho_p = \phi_\mu^p D_{\mu\nu} \phi_\nu^p$$

In [2]:
D = wfn.Da()

rho = []
points_func = Vpot.properties()[0]
superfunc = Vpot.functional()
for b in range(Vpot.nblocks()):
    
    block = Vpot.get_block(b)
    points_func.compute_points(block)
    npoints = block.npoints()
    
    phi = np.array(points_func.basis_values()["PHI"])[:npoints]
    
    rho = np.einsum('pm,mn,pn->p', phi, D, phi)
    print(rho)

[ 0.       0.00046  0.04882  0.57881  1.36051]
[ 0.       0.       0.       0.       0.       0.00046  0.00046  0.00046  0.00046  0.00046  0.04882  0.04882  0.04882  0.04882  0.04882  0.57881  0.57881  0.57881  0.57881  0.57881  1.36051  1.36051
  1.36051  1.36051  1.36051]


## 3. Evaluating the kernel



In [79]:
D = wfn.Da()

rho = []
points_func = Vpot.properties()[0]
superfunc = Vpot.functional()

func = psi4.core.LibXCFunctional("XC_LDA_X", True)
#func = psi4.core.LibXCFunctional("XC_LDA_C_VWN_RPA", True)

RS_FACTOR = 0.6203504908994000166680068120477781673508
for b in range(Vpot.nblocks()):
    
    block = Vpot.get_block(b)
    points_func.compute_points(block)
    npoints = block.npoints()
    
    phi = np.array(points_func.basis_values()["PHI"])[:npoints]
    
    rho = 2.0 * np.einsum('pm,mn,pn->p', phi, D, phi)
    #rho[rho < 1.e-3] = 1.e-3
    print("RHO", rho)
    
    inp = {}
    inp["RHO_A"] = psi4.core.Vector.from_array(rho)
    
    ret = superfunc.compute_functional(inp, -1)
    vk = np.array(ret["V"])[:npoints]
    v_rho_a = 0.5 * np.array(ret["V_RHO_A"])[:npoints]
    
    alpha = 4.0/3.0
    denx = (rho ** (-1.0 /3.0))
    v_x = 0.984745953 * ( -3/4) * rho / denx
    v_rho_x = (-3/4) / denx 
    print("S  ", v_x)
    print("SR ", v_rho_x)
    
    lrs = np.log(denx)
    rpa_c = 0.0311 * lrs - 0.048 + 0.009 * lrs * denx -0.017 * denx
    #print("RPA", rpa_c)
    
    out = {}
    out["V"] = psi4.core.Vector(npoints)
    out["V_RHO_A"] = psi4.core.Vector(npoints)
    
    func.compute_functional(inp, out, npoints, 1)
    print("Sxc", out["V"].np[:npoints])
    print("Sxc", out["V_RHO_A"].np[:npoints])
    
    #print("---\n")
    #print("VK ", np.array(vk))
    print("VR ", np.array(v_rho_a))
    #print(np.array(v_rho_a))
    print("---\n")
    break
    

RHO [ 0.       0.00091  0.09764  1.15763  2.72103]
S   [-0.      -0.00007 -0.0332  -0.89773 -2.80562]
SR  [-0.0004  -0.07269 -0.34535 -0.7875  -1.04706]
Sxc [-0.      -0.00007 -0.0332  -0.89773 -2.80562]
Sxc [-0.00052 -0.09544 -0.45345 -1.03398 -1.37479]
---

VK  [-0.      -0.0001  -0.04022 -1.00551 -3.08011]
VR  [-0.00091 -0.06997 -0.26674 -0.56803 -0.74243]
---



Refs:
- Johnson, B. G.; Fisch M. J.; *J. Chem. Phys.*, **1994**, *100*, 7429